In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
from IPython import display

In [3]:
!pip install segment_anything
display.clear_output()

In [4]:
import os
HOME = os.getcwd()
print("HOME:", HOME)

HOME: /content


In [5]:
%cd {HOME}
!git clone https://github.com/IDEA-Research/GroundingDINO.git
%cd {HOME}/GroundingDINO
!git checkout -q 57535c5a79791cb76e36fdb64975271354f10251
!pip install -q -e .

/content
Cloning into 'GroundingDINO'...
remote: Enumerating objects: 328, done.
remote: Counting objects: 100% (100/100), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 328 (delta 51), reused 56 (delta 38), pack-reused 228
Receiving objects: 100% (328/328), 12.35 MiB | 17.38 MiB/s, done.
Resolving deltas: 100% (156/156), done.
/content/GroundingDINO
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 72.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.9/200.9 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 69.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 63.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 94.1 MB/s eta 0:00:00


In [6]:
%cd {HOME}

import sys
!{sys.executable} -m pip install 'git+https://github.com/facebookresearch/segment-anything.git'

/content
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-tmp2gg20
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-tmp2gg20
  Resolved https://github.com/facebookresearch/segment-anything.git to commit 6fdee8f2727f4506cfbbe553e23b895e27956588
  Preparing metadata (setup.py) ... done


In [7]:
!pip uninstall -y supervision
!pip install -q supervision==0.6.0

import supervision as sv
print(sv.__version__)

Found existing installation: supervision 0.4.0
Uninstalling supervision-0.4.0:
  Successfully uninstalled supervision-0.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
groundingdino 0.1.0 requires supervision==0.4.0, but you have supervision 0.6.0 which is incompatible.
0.6.0


In [8]:
import os
GROUNDING_DINO_CONFIG_PATH = os.path.join(HOME, "GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py")
print(GROUNDING_DINO_CONFIG_PATH, "; exist:", os.path.isfile(GROUNDING_DINO_CONFIG_PATH))

/content/GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py ; exist: True


In [9]:
%cd {HOME}
!mkdir -p {HOME}/weights
%cd {HOME}/weights
!wget -q https://github.com/IDEA-Research/GroundingDINO/releases/download/v0.1.0-alpha/groundingdino_swint_ogc.pth

/content
/content/weights


In [10]:
import os
GROUNDING_DINO_CHECKPOINT_PATH = os.path.join(HOME, "weights", "groundingdino_swint_ogc.pth")
print(GROUNDING_DINO_CHECKPOINT_PATH, "; exist:", os.path.isfile(GROUNDING_DINO_CHECKPOINT_PATH))

/content/weights/groundingdino_swint_ogc.pth ; exist: True


In [11]:
%cd {HOME}
!mkdir -p {HOME}/weights
%cd {HOME}/weights
!wget -q https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth

/content
/content/weights


In [12]:
import os
SAM_CHECKPOINT_PATH = os.path.join(HOME, "weights", "sam_vit_h_4b8939.pth")
print(SAM_CHECKPOINT_PATH, "; exist:", os.path.isfile(SAM_CHECKPOINT_PATH))

/content/weights/sam_vit_h_4b8939.pth ; exist: True


In [13]:
import torch
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [14]:
%cd {HOME}/GroundingDINO
from groundingdino.util.inference import Model
grounding_dino_model = Model(model_config_path=GROUNDING_DINO_CONFIG_PATH, model_checkpoint_path=GROUNDING_DINO_CHECKPOINT_PATH)

/content/GroundingDINO


/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


final text_encoder_type: bert-base-uncased


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [15]:
SAM_ENCODER_VERSION = "vit_h"

In [16]:
from segment_anything import sam_model_registry, SamPredictor
sam = sam_model_registry[SAM_ENCODER_VERSION](checkpoint=SAM_CHECKPOINT_PATH).to(device=DEVICE)
sam_predictor = SamPredictor(sam)

In [17]:
from typing import List
import cv2
import supervision as sv
import numpy as np
from segment_anything import SamPredictor
import math
import glob

In [18]:
from google.colab.patches import cv2_imshow

In [19]:
def enhance_class_name(class_names: List[str]) -> List[str]:
    return [
        f"all {class_name}s"
        for class_name
        in class_names
    ]

In [20]:
def segment(sam_predictor: SamPredictor, image: np.ndarray, xyxy: np.ndarray) -> np.ndarray:
    sam_predictor.set_image(image)
    result_masks = []
    for box in xyxy:
        masks, scores, logits = sam_predictor.predict(
            box=box,
            multimask_output=True
        )
        index = np.argmax(scores)
        result_masks.append(masks[index])
    return np.array(result_masks)

In [21]:
def normalize_coordinates(row_i, col_j, img):
    num_rows, num_cols = img.shape[:2]
    x = col_j/(num_cols - 1.)
    y = row_i/(num_rows - 1.)
    return x, y

In [29]:
def getMask(SOURCE_IMAGE_PATH, CLASSES, BOX_TRESHOLD, TEXT_TRESHOLD, name):
  image = cv2.imread(SOURCE_IMAGE_PATH)
  # image = cv2.resize(image, (640, 640))
  # detect objects
  detections = grounding_dino_model.predict_with_classes(
      image=image,
      classes=enhance_class_name(class_names=CLASSES),
      box_threshold=BOX_TRESHOLD,
      text_threshold=TEXT_TRESHOLD)
  # convert detections to masks
  detections.mask = segment(
      sam_predictor=sam_predictor,
      image=cv2.cvtColor(image, cv2.COLOR_BGR2RGB),
      xyxy=detections.xyxy
  )
  class_map = {'gun': '0', 'bottle': '1', 'glass': '2'}
  maskSS = detections.mask
  box_annotator = sv.BoxAnnotator()
  mask_annotator = sv.MaskAnnotator()
  labels = [
    f"{CLASSES[class_id]} {confidence:0.2f}" 
    for _, _, confidence, class_id, _ 
    in detections]
  print(labels)
  if len(labels) == 0:
    os.remove(SOURCE_IMAGE_PATH)
  else:
    cll = b = [i.split(" ")[0] for i in labels]
    with open("/content/gdrive/My Drive/YOLOV8/labels/" + str(name) + '.txt', 'w') as f:
      for qq, x in enumerate(maskSS):
        w = class_map[cll[qq]]
        im = image
        im[:, :] = [0, 0, 0]
        pixelpoints = np.array(np.nonzero(x))
        pixelpoints = np.array([pixelpoints[1], pixelpoints[0]]).T
        pts = pixelpoints.reshape((-1, 1, 2))
        cv2.polylines(im, pts, True, (255, 255, 255), 2)
        ret, threshed_img = cv2.threshold(cv2.cvtColor(im, cv2.COLOR_BGR2GRAY), 10, 200, cv2.THRESH_BINARY)
        # contours, hier = cv2.findContours(threshed_img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        contours, _ = cv2.findContours(threshed_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        # Convert the contour to the format required for segmentation in COCO format
        # Write bounding boxes to file in YOLO format
        for contour in contours:
            # Get the bounding box coordinates of the contour
            # cv2.drawContours(image, contours, -1, (0,255,0), 3)
            # cv2_imshow(image)
            arr_reshaped = contour.reshape(contour.shape[0], -1)
            segmentation = contour.flatten().tolist()
            # x, y, w, h = cv2.boundingRect(contour)
            # Convert the coordinates to YOLO format and write to file
            partitioned_list = [segmentation[i:i + 2] for i in range(0, len(segmentation), 2)]
            f.write(str(w) +' ')
            for x,y in partitioned_list:
              x, y = normalize_coordinates(y, x, image)
              f.write(str(x) + ' ' + str(y) + ' ')
            f.write("\n")
            break

In [30]:
import glob
for fin in glob.glob("/content/gdrive/My Drive/YOLOV8/images/*"):
  SOURCE_IMAGE_PATH = fin
  print(fin)
  name = SOURCE_IMAGE_PATH.split("/")
  name = name[len(name) - 1].split(".")[0]
  CLASSES = ['gun', 'bottle', 'glass']
  BOX_TRESHOLD = 0.50
  TEXT_TRESHOLD = 0.30
  getMask(SOURCE_IMAGE_PATH, CLASSES, BOX_TRESHOLD, TEXT_TRESHOLD, name)
  print("________________________________________")

/content/gdrive/My Drive/YOLOV8/images/9.jpg


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:866: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


['bottle 0.53']
________________________________________
/content/gdrive/My Drive/YOLOV8/images/7.jpg
[]
________________________________________
/content/gdrive/My Drive/YOLOV8/images/8.jpg
['bottle 0.63']
________________________________________
/content/gdrive/My Drive/YOLOV8/images/6.jpg
['gun 0.75', 'bottle 0.53']
________________________________________
/content/gdrive/My Drive/YOLOV8/images/4.jpg
['gun 0.63']
________________________________________
/content/gdrive/My Drive/YOLOV8/images/2.jpg
['glass 0.52']
________________________________________
/content/gdrive/My Drive/YOLOV8/images/5.jpg
[]
________________________________________
/content/gdrive/My Drive/YOLOV8/images/3.jpg
[]
________________________________________
/content/gdrive/My Drive/YOLOV8/images/10.jpg
[]
________________________________________
/content/gdrive/My Drive/YOLOV8/images/13.jpg
['gun 0.54']
________________________________________
/content/gdrive/My Drive/YOLOV8/images/14.jpg
[]
_____________________